In [29]:
# https://towardsdatascience.com/creating-an-interactive-map-in-python-using-bokeh-and-pandas-f84414536a06

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
import numpy as np
import pandas as pd

In [30]:
sieges_df = pd.read_csv('sieges.csv', index_col=0, encoding='cp1252')

In [31]:
sieges_df.head()

,Siege,Conflict,Year,Casualties,lon,lat
1,Siege of Sidon,Phoenician rebellion against Persia,-343,40000,35.372948,33.557069
2,Siege of Jerusalem,First Jewish–Roman War,70,65000,35.213710,31.768319
3,Siege of Constantinople,Arab–Byzantine wars,717,150000,28.978359,41.008238
4,Siege of Jerusalem,First Crusade,1099,40000,35.213710,31.768319
5,Siege of Baghdad,Mongol conquests,1258,1175000,44.361488,33.312806


In [32]:
# Define function to switch from lat/long to mercator coordinates

def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)

sieges_df['coordinates'] = list(zip(sieges_df['lat'], sieges_df['lon']))

# Obtain list of mercator coordinates

mercators = [x_coord(x, y) for x, y in sieges_df['coordinates'] ]

In [33]:
# Create mercator column in our df

sieges_df['mercator'] = mercators

# Split that column out into two separate columns - mercator_x and mercator_y

sieges_df[['mercator_x', 'mercator_y']] = sieges_df['mercator'].apply(pd.Series)

In [34]:
sieges_df.head()

,Siege,Conflict,Year,Casualties,lon,lat,coordinates,mercator,mercator_x,mercator_y
1,Siege of Sidon,Phoenician rebellion against Persia,-343,40000,35.372948,33.557069,"(33.5570691, 35.372948)","(3937698.559216945, 3969481.16396569)",3.937699e+06,3.969481e+06
2,Siege of Jerusalem,First Jewish–Roman War,70,65000,35.213710,31.768319,"(31.768319, 35.21371)","(3919972.266142005, 3732937.169451003)",3.919972e+06,3.732937e+06
3,Siege of Constantinople,Arab–Byzantine wars,717,150000,28.978359,41.008238,"(41.0082376, 28.9783589)","(3225856.156772727, 5013556.783905713)",3.225856e+06,5.013557e+06
4,Siege of Jerusalem,First Crusade,1099,40000,35.213710,31.768319,"(31.768319, 35.21371)","(3919972.266142005, 3732937.169451003)",3.919972e+06,3.732937e+06
5,Siege of Baghdad,Mongol conquests,1258,1175000,44.361488,33.312806,"(33.3128057, 44.3614875)","(4938298.199332171, 3936897.672595947)",4.938298e+06,3.936898e+06


In [35]:
# Select tile set to use

chosentile = get_provider(Vendors.STAMEN_TONER)

In [52]:
# Choose palette

palette = RdYlGn[11]

In [53]:
# Tell Bokeh to use df as the source of the data

source = ColumnDataSource(data=sieges_df)

In [54]:
# Define color mapper - which column will define the colour of the data points

color_mapper = linear_cmap(field_name = 'Casualties', palette = palette, low = sieges_df['Casualties'].min(), high = sieges_df['Casualties'].max())

In [55]:
# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful

tooltips = [('Casualties','@Casualties'), ('Siege','@Siege'), ('Year','@Year')]

In [56]:
# Create figure

p = figure(title = 'Historical Battles & Sieges by Number of Casualties', x_axis_type="mercator", y_axis_type="mercator", x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

In [57]:
# Add map tile

p.add_tile(chosentile)

TileRenderer(id='1426', ...)

In [58]:
# Add points using mercator coordinates

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=30, fill_alpha = 0.7)

GlyphRenderer(id='1434', ...)

In [59]:
#Defines color bar

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location

p.add_layout(color_bar, 'right')

In [60]:
# Display in notebook

output_notebook()

# Save as HTML

output_file('sieges.html', title='Historical Battles & Sieges by Number of Casualties')

Loading BokehJS ...

In [61]:
# Show map

show(p)